Welcome to week 8, part 1 of the [Noisebridge Python Class](https://github.com/audiodude/PythonClass)!

This class will be a little different. It covers the basics of web app development, using the excellent Flask package. However, Flask can't run inside a Jupyter notebook. Therefore, the different sets of code for this class are inside of the different folders in this directory. The code must be run using a local Python installation and the command line, which we will cover briefly.

[Here](https://python.land/installing-python) are some instructions for installing Python on Windows, macOS, or Linux.

In this lesson, you will learn:

* Setting up a Python development environment with virtualenv
* A basic "hello world" Flask app and how to run it
* Using HTML templates with your Flask app, including template inheritance

Part 2 will cover building a functional web app from scratch:

* *Setting up an SQL schema for use with the app (repeat of SQL lesson)*
* *Receiving input from the URL and URL matching*
* *Receiving user input from a `<form>` and creating a new row in a database*
* *Using templating to display a list of the data added to the app*

Let's get started!

First, we must set up a [virtualenv](https://docs.python.org/3/library/venv.html) for our code. A virtualenv allows us to install certain Python libraries/packages in an isolated environment. Suppose you're working on multiple Python projects on your computer. You don't want the libraries from one app to be included in a different app, especially if they have different required versions.

We can create a virtualenv very simply in Python 3. First we create a directory for our app (this has already been done in this repo):

```
mkdir intro-flask
```

Then we switch into that directory and create a virtualenv (this is required, because the virtualenv is never checked into source control directly):

```
cd intro-flask
python -m venv venv
```

In case you're wondering, the `-m venv` argument tells python to run the `venv` module, and the first and only argument to that module is also the string `venv`.

Then we can activate our virtualenv:

```
source venv/bin/activate
```

We must repeat this command in every new shell/command/terminal window once we are inside the `intro-flask` directory, to make sure we are using our virtualenv. You might notice that your shell prompt now starts with `(venv)` to tell you that you're using the virtualenv. If you need to stop using it for any reason (maybe to start using a different one) you can use simply use the command:

```
deactivate
```

Once we've created and activated our virtualenv, we can install the `flask` package using `pip`, the Python package manager. The `pip` command will automatically install any additional packages that are needed for the package you requested:

```bash
pip install flask
```

You should see a bunch of text fly by as Python downloads and installs the necessary package(s).

Finally, it is common to save a snapshot of the packages you are using, so that others who wish to use your code, or perhaps yourself in the future, can easily re-install the exact versions that were known to be working with your code. To do this, we use `pip freeze`. First, run `pip freeze` on its own, to see the output:

```
$ pip freeze
blinker==1.6.2
click==8.1.6
Flask==2.3.2
importlib-metadata==6.8.0
itsdangerous==2.1.2
Jinja2==3.1.2
MarkupSafe==2.1.3
Werkzeug==2.3.6
zipp==3.16.2
```

Then you can redirect the output of the command to a file (a common command line/bash operation). We will call this file `requirements.txt` as is common in the Python world.

```
pip freeze > requirements.txt
cat requirements.txt
```

When we run `cat` on requirements.txt, we see it's content, which should be the same as when we ran `pip freeze`.

When you need to install your requirements again, use the `-r` flag to pip install:

```
pip install -r requirements.txt
```

---

It is finally time to write our first "hello world" web app. In Flask, this is as simple as the following code, which you can find in `intro-flask/01-hello-world/main.py`. *Note that this code will do nothing in the Jupyter notebook.*

In [1]:
import flask

app = flask.Flask(__name__)

@app.route('/')
def index():
    return 'Hello World!'

If you recall from the lesson on web scraping, web sites run from applications known as **servers**. The server is always running, and waits for a **request** from a client, usually a web browser. When the server receives a request, it responds with whatever it's been programmed or configured to send back to the client. In this example, the server will always send back the plain text `Hello World!`.

The code above imports the flask library, then creates a new Flask app using `__name__` as its name, where `__name__` is a special Python variable which represents the name of the currently running module.

Next, the code uses the **decorator** `@app.route('/')` to modify the function right below it (`def index():`). This decorator tells Flask that the `index` function should be called when a request is made for the server path `/`, which is the root of the server and the path clients request when they visit the website without any path specified. So, for example, when you request `https://en.wikipedia.org` in your web browser, you are actually requesting the path `/` on the server `en.wikipedia.org`.

Let's start our server. First change into the directory containing the above code:

```
cd intro-flask/01-hello-world
```

Then, we use the `flask` command to start an HTTP server with our hello world Flask app:

```
FLASK_APP=main flask run
```

The portion `FLASK_APP=main` sets an [environment variable](https://www.digitalocean.com/community/tutorials/how-to-read-and-set-environmental-and-shell-variables-on-linux) which is a variable that is availble to all code running in the current shell. Since we specified it immediately before our command, it is only set and active for that one command.

The portion `flask run` starts the `flask` command with the argument `run`. The `flask` command is available in your environment via pip and the virtualenv you installed. If you get an error like `flask: command not found`, it is likely that your virtualenv is not set up correctly or you haven't installed flask. 

If your `flask` command runs successfully, you should see the following:

```
$ FLASK_APP=main flask run
 * Serving Flask app 'main'
 * Debug mode: off
WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
```

Now let's open a web browser and check out our new web app! Open a new browser window and use either the URL that the Flask printed out (`http://127.0.0.1:5000`) or the equivalent shortcut `http://localhost:5000`. Web app developers generally use `localhost` because it is shorter and easier to type, but it is equivalent to `127.0.0.1`.

You should see:

![Screenshot of web app display 'Hello World!' in a browser window](https://pixelfed.social/storage/m/_v2/588554065884192073/c51ea9690-856390/F9rl3ISqzW8l/BgUbbihwvrj2QakNzCUjOF75ODDFv54iJRaPWayl.png)

The text in the browser is the text that we setup in our `index` function.

You may also have noticed some print out in your terminal:

```
127.0.0.1 - - [06/Aug/2023 09:42:03] "GET / HTTP/1.1" 200
```

This a **log** of the request that was made by your browser, and indicates that a successful response of 200 was sent back (recall from a previous lesson that 200 represents success in HTTP).

---

Recall that the "source code" of the web is HTML. HTML is represented as **documents** which are served to the browser. In some cases, you can have a **static** HTML document, that exists as a file on the server. So when you go to `https://www.nasa.gov/kidsclub/index.html`, the server looks for a file called `index.html` in the `kidsclub` directory and serves it directly to the browser.

However, for web apps, we create HTML documents **dynamically**. The document doesn't exist as a file. In fact, it doesn't exist at all until the browser request is handled by the server. Then, the server constructs the document and returns it to the browser. This is how, for example, all Wordpress sites operate (except that the programming language used is [PHP](https://www.php.net/) not Python).

For our "hello world" example, we didn't return an HTML document. We returned plain text, which the browser interpreted and displayed appropriately. The basic skeleton of an HTML document is the following:

```
<!DOCTYPE html>
<html>
<head>
  <title>Hello world website</title>
</head>
<body>
  Hello World!
</body>
</html>
```

The `<!DOCTYPE` declaration informs the browser which version of HTML we are using. In modern times, we always use `<!DOCTYPE html>` to indicate the latest version of HTML.

The `<body>` tag is where we put the content that will be displayed in the browser.

What if we didn't want to always say hello, but also wanted to print the current time?

In [2]:
import datetime

import flask

app = flask.Flask(__name__)

@app.route('/')
def index():
    return 'Hello World! ' + str(datetime.datetime.now())

![Screenshot of hello world application with time displayed](https://pixelfed.social/storage/m/_v2/588554065884192073/c51ea9690-856390/sXxD1TtoRkVL/C1owtRyIAo6snhgptYaJh8zPzHmfzCRKnyPebJPk.png)

Now our hello world web app is truly dynamic, it changes every time the page is refreshed!

Imagine though, that we want to serve an HTML version of this data. We could use string formatting to achieve the result.

In [3]:
import datetime

import flask

app = flask.Flask(__name__)

@app.route('/')
def index():
  return ('''<!DOCTYPE html>
    <html>
    <head>
      <title>Hello World website</title>
    </head>
    <body>
      Hello World! %s
    </body>
    </html>''' % str(datetime.datetime.now()))

*(See this example in 02-hello-datetime)*

But what if we also wanted to specify a "Goodbye" page? The most obvious way is the following (where we use a `@app.route('/goodbye')` decorator to indicate that the Goodbye should be shown when the url `/goodbye` is requested):

In [ ]:
import flask

app = flask.Flask(__name__)

@app.route('/')
def index():
  return ('''<!DOCTYPE html>
    <html>
    <head>
      <title>Hello World website</title>
    </head>
    <body>
      Hello World! %s
    </body>
    </html>''' % str(datetime.datetime.now()))

@app.route('/goodbye')
def goodbye():
  return ('''<!DOCTYPE html>
    <html>
    <head>
      <title>Hello World website</title>
    </head>
    <body>
      Goodbye! %s
    </body>
    </html>''' % str(datetime.datetime.now()))

![Screenshot of web app showing Goodbye and the time](https://pixelfed.social/storage/m/_v2/588554065884192073/c51ea9690-856390/Nk5YduPx9ZBX/O10nIHwVaTa3UkGsneBEvg1DlTKLx04n6Do14cY8.png)

We see now that we have a lot of duplicate code! Maybe we can refactor it a bit:

In [4]:
import datetime

import flask

app = flask.Flask(__name__)

def get_document(greeting, time_value):
  return f'''<!DOCTYPE html>
    <html>
    <head>
      <title>Hello World website</title>
    </head>
    <body>
      {greeting} {time_value}
    </body>
    </html>'''

@app.route('/')
def index():
  return get_document('Hello World!', datetime.datetime.now())

@app.route('/goodbye')
def goodbye():
  return get_document('Goodbye!', datetime.datetime.now())

*(See example in 03-inferior-templating)*

This is the idea behind **templates** or **templating**. Most of the HTML document stays the same, but we have a few sections that we'd like to specify as variables. Luckily, Flask supports a powerful templating system, using [Jinja2](https://jinja.palletsprojects.com/en/3.1.x/). We can use Flask's templating system to keep our HTML templates in their own files, separate from our code, which makes them easier to edit and keeps our app logic cleaner.

First, we create a directory alongside our main module called `templates`:

```
mkdir templates
```

Then we create a file inside the templates directory, named `greeting.html`:

```
<!DOCTYPE html>
<html>
  <head>
    <title>Hello World website</title>
  </head>
  <body>
    {{ greeting }} {{ time_value }}
  </body>
</html>
```

The name of the template doesn't matter, we will refer to it directly in code.

Finally we update our application logic:

In [5]:
import datetime

import flask

app = flask.Flask(__name__)

@app.route('/')
def index():
  return flask.render_template(
    'greeting.html',
    greeting='Hello World!',
    time_value=datetime.datetime.now())

@app.route('/goodbye')
def goodbye():
  return flask.render_template(
    'greeting.html',
    greeting='Goodbye!',
    time_value=datetime.datetime.now())

This is great, now our app logic is very simple and clean, and all of the HTML boilerplate that is repeated between the two documents is isolated in the template.

Part 2 will cover creating a functional web app!